# LAVA Data Value
## For Mislabeled Data Detection (STL10)

In [1]:
import lava

In [2]:
import torch
import torchvision
# print(torch.__version__)
# print(torchvision.__version__)
# print(torch.version.cuda)
# print(torch.cuda.is_available())

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.models as models
from torch.autograd import Variable

import matplotlib.pyplot as plt
from torch import tensor
from torchvision import datasets, transforms
import pandas as pd
import numpy as n

from torch.utils.data import Dataset, TensorDataset, DataLoader

In [ ]:
! nvidia-smi

In [ ]:
cuda_num = 0
import torchvision
print(torchvision.__version__)
import torch
print(torch.__version__)
import os
os.environ["CUDA_VISIBLE_DEVICES"]=str(cuda_num)
print(os.environ["CUDA_VISIBLE_DEVICES"])
torch.cuda.set_device(cuda_num)
print("Cuda device: ", torch.cuda.current_device())
print("cude devices: ", torch.cuda.device_count())
device = torch.device('cuda:' + str(cuda_num) if torch.cuda.is_available() else 'cpu')

### Set Parameters

In [5]:
training_size = 5000
valid_size = 2000
resize = 32
portion = 0.3

### Loading Corrupted Data into DataLoaders
### Train and Test Loaders with Indices of Corrupted Data

In [6]:
loaders, shuffle_ind = lava.load_data_corrupted(corrupt_type='shuffle', dataname='STL10', resize=resize,
                                        training_size=training_size, test_size=valid_size, currupt_por=portion)

### Load Feature Extractor for a Given Dataset
### Checkpoints provided and training files provided

In [7]:
feature_extractor = lava.load_pretrained_feature_extractor('preresnet18_test_stl10.pth', device)

### Compute Dual Solution of Optimal Transport Problem

In [8]:
dual_sol, trained_with_flag = lava.compute_dual(feature_extractor, loaders['train'], loaders['test'], 
                                                training_size, shuffle_ind, resize=resize)

### Compute the Data Values from Calibrated Gradient
### Visualize Detection Rate Based on LAVA

In [9]:
%matplotlib inline
calibrated_gradient = lava.compute_values_and_visualize(dual_sol, trained_with_flag, training_size, portion)

In [10]:
calibrated_gradient